Build a function that identifies variables that correlate w/ protected classes to mitigate risk of unintentional biases in our HR models

In [29]:
import pandas as pd
import math
import os
import sys
from pathlib import Path
import csv
import seaborn as sn
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
os.chdir('C:/Users/Bridget Marie Yoga/Documents/GitPy/Python')

In [5]:
hr = pd.read_csv('HRDataset_v14.csv')

Let's explore the data!

In [6]:
hr.describe()

,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Zip,ManagerID,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences
count,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,311.000000,303.000000,311.000000,311.000000,311.000000,311.000000,311.000000
mean,10156.000000,0.398714,0.810289,0.434084,2.392283,4.610932,2.977492,0.093248,69020.684887,0.334405,16.845659,6555.482315,14.570957,4.110000,3.890675,1.218650,0.414791,10.237942
std,89.922189,0.490423,0.943239,0.496435,1.794383,1.083487,0.587072,0.291248,25156.636930,0.472542,6.223419,16908.396884,8.078306,0.789938,0.909241,2.349421,1.294519,5.852596
min,10001.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,45046.000000,0.000000,1.000000,1013.000000,1.000000,1.120000,1.000000,0.000000,0.000000,1.000000
25%,10078.500000,0.000000,0.000000,0.000000,1.000000,5.000000,3.000000,0.000000,55501.500000,0.000000,18.000000,1901.500000,10.000000,3.690000,3.000000,0.000000,0.000000,5.000000
50%,10156.000000,0.000000,1.000000,0.000000,1.000000,5.000000,3.000000,0.000000,62810.000000,0.000000,19.000000,2132.000000,15.000000,4.280000,4.000000,0.000000,0.000000,10.000000
75%,10233.500000,1.000000,1.000000,1.000000,5.000000,5.000000,3.000000,0.000000,72036.000000,1.000000,20.000000,2355.000000,19.000000,4.700000,5.000000,0.000000,0.000000,15.000000
max,10311.000000,1.000000,4.000000,1.000000,5.000000,6.000000,4.000000,1.000000,250000.000000,1.000000,30.000000,98052.000000,39.000000,5.000000,5.000000,8.000000,6.000000,20.000000


In [51]:
hr.dtypes

Employee_Name                  object
EmpID                           int64
MarriedID                       int64
MaritalStatusID                 int64
GenderID                        int64
EmpStatusID                     int64
DeptID                          int64
PerfScoreID                     int64
FromDiversityJobFairID          int64
Salary                          int64
Termd                           int64
PositionID                      int64
Position                       object
State                          object
Zip                             int64
DOB                            object
Sex                            object
MaritalDesc                    object
CitizenDesc                    object
HispanicLatino                 object
RaceDesc                       object
DateofHire                     object
DateofTermination              object
TermReason                     object
EmploymentStatus               object
Department                     object
ManagerName 

In [10]:
hr.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,...,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,...,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,...,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,...,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,...,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2


In [88]:
#Encode for correlation
for i in hr.columns:
    if hr[i].dtype == object:
        hr[i]=hr[i].astype('category').cat.codes

In [91]:
mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
tri_df = corrMatrix.mask(mask)

# List column names of highly correlated features (r > 0.95)
to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.3)]

Look at correlations w/ sensitive variables

In [97]:
tri_df['GenderID'].abs().sort_values(ascending=False).head()

Sex                     1.000000
ManagerName             0.094303
Position                0.093812
SpecialProjectsCount    0.087073
PositionID              0.081612
Name: GenderID, dtype: float64

In [118]:
tri_df['RaceDesc'].abs().sort_values(ascending=False).head()

RecruitmentSource             0.100009
SpecialProjectsCount          0.056448
EngagementSurvey              0.053692
TermReason                    0.044680
LastPerformanceReview_Date    0.035014
Name: RaceDesc, dtype: float64

In [100]:
tri_df['HispanicLatino'].abs().sort_values(ascending=False).head()

DateofHire          0.112564
Department          0.106760
EngagementSurvey    0.072865
EmpSatisfaction     0.056722
DaysLateLast30      0.052670
Name: HispanicLatino, dtype: float64

Make it a function

In [120]:
def corr_var(df, var, threshold):
    for i in df.columns:
        if df[i].dtype == object:
            df[i]=df[i].astype('category').cat.codes
    
    corrMatrix = df.corr()
    mask = np.triu(np.ones_like(corrMatrix, dtype=bool))
    tri_df = corrMatrix.mask(mask)
    tri_df[var] = tri_df[var].abs().sort_values(ascending=False)
    
    # List column names of highly correlated features (r > threshold)    
    to_drop = [c for c in tri_df.columns if any(tri_df[c] > threshold)]
    return to_drop
    
    
    

In [121]:
corr_var(hr,'GenderID',0.7)

['GenderID',
 'EmpStatusID',
 'DeptID',
 'Termd',
 'PositionID',
 'DateofTermination',
 'PerformanceScore']